### **Example 4**


#### Stage 1: Job Script: `example-four-stage1.sh`

In [ ]:
%%file "example-four-stage1.sh"
#!/bin/bash
## Generic Info
#SBATCH --job-name "Stage 1: Example 4"  # job name
#SBATCH --output   "logs/stage1.%j.log"  # output file pattern

# create virtual environment
python3 -m venv "env"

# activate environment
source "env/bin/activate"

# install packages into environment
python3 -m pip install numpy

# With python environment setup,
#    ...  we request "stage 2" be scheduled
sbatch "example-four-stage2.sh"

#### Stage 2: Job Script: `example-four-stage2.sh`

In [ ]:
%%file "example-four-stage2.sh"
#!/bin/bash
## Request Job-Array
#SBATCH --array 1-20%10 # the array has 100 sub-jobs (labeled 1 to 100) 
                        # .. with at most 10 sub-jobs running at 
                        # .. any given point.

## Generic Info for "sub-job"
#SBATCH --job-name "Stage 2: Example 4"    # sub-job name
#SBATCH --output   "logs/stage2.%A.%a.log" # output file pattern
                                           # .. %A -> array-job-id
                                           # .. %a -> array-task-id 

## Resources To Request For Each "sub-job"
#SBATCH --ntasks 1
#SBATCH --cpus-per-task 1
#SBATCH --mem-per-cpu 500
#SBATCH --time 00:45:00  

## Commands Executed By Each "sub-job"
source "env/bin/activate"         # activate environment from "stage 1"
python3 "scripts/example-four.py" # execute python script

#### Stage 2: Python Script: `example-four.py`

```python
"""
 File: `example-four.py`
 Synopsis:
    estimate condition number and print results for 
  a single random perturbation of the nxn identity matrix 
  (where n is fixed).
"""
from numpy import eye
from numpy.random import uniform
from numpy.linalg import cond

n = 1000
eps = 0.01

I = eye(n) # nxn identity matix
X = uniform(size=[n,n])

# generate perturbation `Z`
Z = I + eps*uniform()*X 

# estimate condition number of `Z`
c = cond(Z)

# print results
print(f'{c}', flush=True)
```

#### Submiting & Results

To see what's happening we'll need to switch to a terminal and use the following script.

In [ ]:
%%file "watch-queue.sh"
#!/bin/bash

# submit seed job (stage1)
sbatch "example-four-stage1.sh"

# watch queue
ii=0 
while [ $ii -lt 1000 ]; do
    squeue --user $USER --name "Stage 1: Example 4","Stage 2: Example 4"
    sleep 0.7 && clear
    ii=$((ii+1))
done